In [ ]:
import sys
import numpy as np
import pandas as pd
sys.path.append('../')

from config.paths import PROCESSED_DATA_PATH

from config.paths import PROCESSED_DATA_PATH
from utils.data_split import temporal_train_test_split

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


In [ ]:
processed_data_path = PROCESSED_DATA_PATH / "processed_data.parquet"
df = pd.read_parquet(processed_data_path)

# Train Test Split

In [ ]:
test_size = 0.2
train_df, test_df = temporal_train_test_split(df, test_size=test_size)

# Model

In [ ]:
# 1. Define el formato de los datos
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[['customer_id', 'movie_id', 'rating']], reader)

# 2. Divide en train/test (aquí podrías usar tu test original también)
trainset = data.build_full_trainset()
testset = list(zip(test_df['customer_id'], test_df['movie_id'], test_df['rating']))

# 3. Entrena el modelo
model = SVD()
model.fit(trainset)

# 4. Evalúa
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
